# Проект: Исследование надежности заемщиков

<font size='4'>**Описание проекта**</font>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

***Цель проекта:*** провести предобработку данных и первоначальный исследовательский анализ данных. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***Задачи проекта:***
- Загрузка и первоначальный анализ данных
- Предобработка данных
- Ответ на вопросы на основе исследовательского анализа данных

<font size='4'>**Необходимые библиотеки**</font>

In [1]:
import pandas as pd

## Загрузка данных и первичный анализ

На данном шаге производится загрузка данных из источника Яндекса, а также первичный анализ загруженных данных для определения последующих шагов предобработки

Загружаем данные из csv файла. Путь к файлу `https://code.s3.yandex.net/datasets/data.csv`

In [2]:
# Используем конструкцию try-except для обработки ошибок при некорректном адрессе файла
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Проверим корректность чтения файла, выведем случайные 20 строчек фрейма

In [3]:
data.sample(20,
            random_state=78)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1950,0,NaN,62,среднее,1,женат / замужем,0,M,компаньон,0,NaN,операции со своей недвижимостью
14648,1,-1622.830092,32,среднее,1,гражданский брак,1,F,сотрудник,0,60518.820000,заняться высшим образованием
10509,2,-2636.753966,46,среднее,1,женат / замужем,0,M,сотрудник,0,125600.372094,покупка коммерческой недвижимости
17245,1,-3754.962527,55,среднее,1,женат / замужем,0,M,компаньон,1,48672.285854,на покупку автомобиля
1915,0,373836.542882,67,высшее,0,вдовец / вдова,2,F,пенсионер,0,230877.037643,недвижимость
13378,0,-778.918807,26,среднее,1,гражданский брак,1,M,сотрудник,0,254834.443525,на проведение свадьбы
4823,0,-583.463940,39,среднее,1,гражданский брак,1,F,сотрудник,0,173221.613498,жилье
3891,1,NaN,68,НЕОКОНЧЕННОЕ ВЫСШЕЕ,2,в разводе,3,F,сотрудник,0,NaN,строительство недвижимости
20349,0,-1680.289676,44,среднее,1,женат / замужем,0,F,сотрудник,0,88406.642907,строительство жилой недвижимости
20121,0,-1733.626977,27,Среднее,1,гражданский брак,1,M,сотрудник,0,99939.720476,сыграть свадьбу


Можно сделать следующие замечания по данным:
- Есть отрицательные значения в признаке `days_employed` - <span id='abs'>эти значения необходимо привести к строго положительным (с помощью функции модуля `abs()`)
- Есть неявные дубликаты в категориальном признаке `education` - эти значения необходимо заменить на общие (например, значение `СРЕДНЕЕ` заменим на `среднее`)
- Есть неявные дубликаты в признаке `purpose` - следует объединить под одним названием (по аналогии с признаком `education`)
- <span id='empty'>Есть пустоты в признаках `total_income` и `days_employed` - следует их либо заполнить, либо удалить

Выведем общую информацию о фрейме

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Таким образом, в данном фрейме мы имеем фрейм из 12 признаков (5 категориальных, 7 количественных) и 21525 объектов. Два [вышеобозначенных](#empty) признака содержат пустоты, которые необходимо обработать далее

## Предобработка данных

### Удаление пропусков

Начнем с заполнения пропусков в признаке `total_income`. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

Перед заполнением пустот в `days_employed` необходимо обработать аномалии (по описанию [выше](#abs))

In [6]:
data['days_employed'] = data['days_employed'].abs()

Теперь проведем аналогичную замену для признака `days_employed`

In [7]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Проверим заполнение пустот в указанных признаках

In [8]:
data[['total_income', 'days_employed']].isna().sum()

total_income     0
days_employed    0
dtype: int64

Заполнение пропусков прошло успешно. Перейдем к обработке выбросов

### Обработка аномальных значений

Начнем с рассмотрения признака `days_employed`. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях

In [9]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправление проведем позже

Выведем перечень уникальных значений столбца `children`

In [10]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В признаке `children` есть два аномальных значения. Удалим объекты, в которых встречаются такие аномальные значения

In [11]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

# Проверка
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Удаление прошло успешно

### Изменение типов данных

Приведем признак `total_income` к типу int

In [12]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в признаке `education`. В этом признаке есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [13]:
data['education'] = data['education'].str.lower()

Выведем число полных дубликатов фрейма

In [14]:
data.duplicated().sum()

71

Удалаим данные значения

In [15]:
data = data.drop_duplicates()
# Проверка
data.duplicated().sum()

0

### Категоризация данных

Создадим признак `total_income_category` с категориями на основе признака `total_income`:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`

Для этого напишем функцию для этого

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

Применим эту функцию к признаку `total_income`

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)
# Проверка
data['total_income_category'].value_counts()

total_income_category
C    15921
B     5014
D      349
A       25
E       22
Name: count, dtype: int64

Выведем на экран перечень уникальных целей взятия кредита из признака `purpose`

In [18]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`

In [19]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [20]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)
# Проверка
data['purpose_category'].value_counts()

purpose_category
операции с недвижимостью    10751
операции с автомобилем       4279
получение образования        3988
проведение свадьбы           2313
Name: count, dtype: int64

### Исследование данные и ответы на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [21]:
# Рассчитаем общее число плательщиков
debt_total = data['debt'].count()

# Найдем общее число людей в каждом сегменте
debt_children_grouped_total = data.groupby('children')['debt'].count()

# Рассчитаем репрезантивность каждой категории
debt_children_repres = debt_children_grouped_total / debt_total
print(debt_children_repres)

children
0    0.660588
1    0.225400
2    0.096198
3    0.015470
4    0.001922
5    0.000422
Name: debt, dtype: float64


Видно, что для нашего анализа подходят не все группы (репрезентативность категории плательщиков, у которых более двух детей,
не подходит для анализа)

In [22]:
# Выберем только строки, где количество детей менее 3х
data_filtered_children = data[data['children'] < 3]

# Рассчитаем количество неуплаченных в срок кредитов по количеству детей
debt_children_grouped_unpayed = data_filtered_children.groupby('children')['debt'].sum()

# Найдем общее число людей в каждом сегменте
debt_children_grouped_total = data_filtered_children.groupby('children')['debt'].count()

# Поделим значения debt_grouped_unpayed, чтобы найти соотношение неоплаченных в срок к общему числу
debt_children_grouped_ratio = debt_children_grouped_unpayed / debt_children_grouped_total

# Выведем на экран результат
print(debt_children_grouped_ratio.sort_values())

children
0    0.075438
1    0.092346
2    0.094542
Name: debt, dtype: float64


**Вывод:** четко выраженной зависимости между количеством детей и возвратом кредита в срок не наблюдается. На репрезентативных категориях видна зависимость между количеством детей и возвратом кредита в срок (прямая зависимость: чем больше детей, тем выше вероятность невозврата кредита в срок)

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
# Рассчитаем количество неуплаченных в срок кредитов по семейному статусу
debt_family_grouped_unpayed = data.groupby('family_status')['debt'].sum()

# Найдем общее число людей в каждом сегменте
debt_family_grouped_total = data.groupby('family_status')['debt'].count()

# Поделим значения debt_family_grouped_unpayed, чтобы найти соотношение неоплаченных в срок к общему числу
debt_family_grouped_ratio = debt_family_grouped_unpayed / debt_family_grouped_total

# Выведем на экран отсортированный результат
print(debt_family_grouped_ratio.sort_values())

family_status
вдовец / вдова           0.066246
в разводе                0.070648
женат / замужем          0.075606
гражданский брак         0.093130
Не женат / не замужем    0.097639
Name: debt, dtype: float64


**Вывод:** наблюдается связь между семейным положением и возвратом кредита в срок, а именно:
те, кто ранее состоял в браке (вдовцы/вдовы или разведенные) наиболее часто возвращают кредиты в срок, наименее часто возвращают те, кто не состоит в официальном браке (те, кто состоят в гражданском браке и неженатые/незамужние)

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
# Найдем общее число людей в каждом сегменте
debt_income_grouped_total = data.groupby('total_income_category')['debt'].count()

# Рассчитаем репрезентативность каждой категории
debt_income_repres = debt_income_grouped_total / debt_total
print(debt_income_repres)

total_income_category
A    0.001172
B    0.235057
C    0.746379
D    0.016361
E    0.001031
Name: debt, dtype: float64


Таким образом, для анализа подойдут только две категории: В и С

In [25]:
# Выберем только строки, где категория дохода В или С
data_filtered_income = data[(data['total_income_category'] == "B") | (data['total_income_category'] == "C")]

# Рассчитаем количество неуплаченных в срок кредитов по категории дохода (которую мы получили в задании 16)
debt_income_grouped_unpayed = data_filtered_income.groupby('total_income_category')['debt'].sum()

# Найдем общее число людей в каждом сегменте
debt_income_grouped_total = data_filtered_income.groupby('total_income_category')['debt'].count()

# Поделим значения debt_income_grouped_unpayed, чтобы найти соотношение неоплаченных в срок к общему числу
debt_income_grouped_ratio = debt_income_grouped_unpayed / debt_income_grouped_total

# Выведем на экран отсортированный результат
print(debt_income_grouped_ratio.sort_values())

total_income_category
B    0.070602
C    0.084982
Name: debt, dtype: float64


**Вывод:** зависимости между уровнем дохода и возвратом кредита в срок нет. Видна зависимость между доходом и возвратом кредита в срок, однако в выборке участвуют только две категории, поэтому нельзя заключить, что данная зависимость прослеживается на полном наборе данных

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [26]:
# Рассчитаем количество неуплаченных в срок кредитов по категории цели (которую мы получили в задании 18)
debt_purpose_grouped_unpayed = data.groupby('purpose_category')['debt'].sum()

# Найдем общее число людей в каждом сегменте
debt_purpose_grouped_total = data.groupby('purpose_category')['debt'].count()

# Поделим значения debt_purpose_grouped_unpayed, чтобы найти соотношение неоплаченных в срок к общему числу
debt_purpose_grouped_ratio = debt_purpose_grouped_unpayed / debt_purpose_grouped_total

# Выведем на экран отсортированный результат
print(debt_purpose_grouped_ratio.sort_values())

purpose_category
операции с недвижимостью    0.072551
проведение свадьбы          0.079118
получение образования       0.092528
операции с автомобилем      0.093480
Name: debt, dtype: float64


**Вывод:** наиболее часто в срок возвращаются кредиты с целью приобритения недвижимости. Наименее часто возвращают кредиты в срок с целью покупки автомобиля.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* пустые значения наблюдались в столбцах days_employed (срок трудоустройства в днях) и total_income (общий доход). Это может быть связано с доступом к информации (информация была скрыта или утеряна) или с технической поломкой (записи были утеряны при копировании).

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* количественные переменные могут содержать пиковые значения (слишком большие или слишком маленькие значения). Такие значения смешают значения в одну из сторон и дают неверное представление о данных. Медиана позволяет избежать такой ошибки

### Общий вывод.

**Вывод:**

Были проанализированные данные о возврате кредитов в срок и других данных плательщика.
Были найдеты и исправлены аномалии в колонке "children" (убраны значения -1 и 20).
Были найдены пустые значения в колонках "total_income" и "days_employed". Заменены на медианные значения соответствующих колонок.
Также были добавлены категории целей кредита и категории дохода.
Были проанализированы взаимосвязи между возвратом кредита в срок и другими данными о налогоплательщике. Была выявлена связь между семейным положением и возвратом кредита в срок, количеством детей и возвратом кредита в срок.